<h1> High Quality Momentum Trading Strategy </h1>

Calculating the number of stocks to be bought for the top 20 High Quality Momentum stocks in the S&P 500 Index for a user specific portfolio.

In [171]:
#Importing libraries

import numpy as np 
import pandas as pd 
import requests
from scipy import stats
import math

#Importing the constituents of the S&P 500

stocks= pd.read_csv('constituents.csv')

#Viewing the constituents

stocks.head()

,Ticker,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


In [172]:
# Acquiring the IEX API Token
from secrets import IEX_CLOUD_API_TOKEN

<h1> Making API Calls and Creating our dataframe</h1>

In [173]:
#We use batch API calls here to improbbe the performance of our code
#reference https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One Year Price Return', 'Number of Shares to Buy']


<h1>High Quality Momentum Trading strategy</h1>

To have an effective Moment Trading Strategy, we need to ensure that the stocks we plan to invest in are High Quality Momentum stocks. High Quality Momentum or HQM stocks are preffered over Low Quality Momentum stocks as they show a steady growth momentum over a larger period of time and are not triggered by rare and specific incidents.

In [174]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Stocks', 
                'One Year Price Return', 
                'One Year Return Percentile',
                'Six Month Price Return',
                'Six Month Return Percentile',
                'Three Month Price Return',
                'Three Month Return Percentile',
                'One Month Price Return',
                'One Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)


<h1>Calculating the percentile scores</h1>
In this part, we will be calculating the momentum percentiles of the enlisted stocks over the periods of:
<ul>
<li>One year price return</li>
<li>Six month price return</li>
<li>Three month price return</li>
<li>One month price return</li>
</ul>

In [175]:
time_periods = [
                'One Year',
                'Six Month',
                'Three Month',
                'One Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
   
hqm_dataframe

,Ticker,Price,Number of Stocks,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,MMM,151.01,N/A,-0.235845,0.13,-0.179675,0.33,-0.149717,0.17,-0.043333,0.54,N/A
1,AOS,62.79,N/A,-0.065196,0.43,-0.013589,0.58,-0.170059,0.12,-0.062347,0.47,N/A
2,ABT,123.87,N/A,-0.026641,0.47,-0.082413,0.5,-0.027903,0.44,-0.01552,0.72,N/A
3,ABBV,157.31,N/A,0.484643,0.96,0.489792,0.98,0.170576,0.9,-0.014751,0.73,N/A
4,ABMD,296.48,N/A,-0.193051,0.23,-0.196297,0.26,-0.000559,0.53,-0.133412,0.23,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CE,148.49,N/A,-0.090814,0.39,-0.136147,0.38,-0.119589,0.25,-0.023347,0.66,N/A
96,CNC,82.54,N/A,0.318515,0.9,0.144892,0.82,0.060228,0.73,-0.057422,0.48,N/A
97,CNP,31.94,N/A,0.361782,0.93,0.211836,0.9,0.155492,0.87,0.031715,0.84,N/A
98,CDAY,58.48,N/A,-0.420622,0.02,-0.576541,0.01,-0.184504,0.09,-0.173662,0.11,N/A


<h1>Calculating the HQM Score</h1> 

The High Quality Momentum score is the arithmetic mean of the afforementioned four momentum percentiles.

In [176]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

# Selecting our top 20 HQM stocks

hqm_dataframe.sort_values(by = 'HQM Score', ascending= True)
hqm_dataframe = hqm_dataframe[:21]


<h1> Accepting the desired portfolio amount and issuing reccomendations</h1>

Here, we accept the user's amount to be invested and return the number of stocks the user should buy of each respective High Quality Momentum Stock.


In [177]:
# Asking for the value of portfolio

def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

#Calculating the number of stocks

position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Stocks'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe




1000000


,Ticker,Price,Number of Stocks,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,MMM,151.01,315,-0.235845,0.13,-0.179675,0.33,-0.149717,0.17,-0.043333,0.54,0.2925
1,AOS,62.79,758,-0.065196,0.43,-0.013589,0.58,-0.170059,0.12,-0.062347,0.47,0.4
2,ABT,123.87,384,-0.026641,0.47,-0.082413,0.5,-0.027903,0.44,-0.01552,0.72,0.5325
3,ABBV,157.31,302,0.484643,0.96,0.489792,0.98,0.170576,0.9,-0.014751,0.73,0.8925
4,ABMD,296.48,160,-0.193051,0.23,-0.196297,0.26,-0.000559,0.53,-0.133412,0.23,0.3125
5,ACN,320.68,148,0.057333,0.64,-0.144551,0.37,-0.090745,0.33,-0.075577,0.42,0.44
6,ATVI,77.6,613,-0.166764,0.25,-0.026789,0.54,-0.030719,0.43,-0.03881,0.56,0.445
7,ADM,94.34,504,0.554968,0.98,0.492377,0.99,0.282768,1.0,0.004635,0.76,0.9325
8,ADBE,426.04,111,-0.239932,0.12,-0.378967,0.06,-0.200515,0.06,-0.079854,0.4,0.16
9,AAP,215.83,220,0.170447,0.74,-0.013126,0.59,0.006401,0.57,0.04341,0.92,0.705
